In [5]:
#conda install -c conda-forge scikit-surprise #adnaconda 프롬프트에서 실행

  Using cached scikit-surprise-1.1.1.tar.gz (11.8 MB)
  Running setup.py clean for scikit-surprise
Failed to build scikit-surprise
    Running setup.py install for scikit-surprise: started
    Running setup.py install for scikit-surprise: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\CJH\anaconda3\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\Public\\Documents\\ESTsoft\\CreatorTemp\\pip-install-x2xpk4ne\\scikit-surprise_73cc58ddbebb4279b6652d98bb1d3e37\\setup.py'"'"'; __file__='"'"'C:\\Users\\Public\\Documents\\ESTsoft\\CreatorTemp\\pip-install-x2xpk4ne\\scikit-surprise_73cc58ddbebb4279b6652d98bb1d3e37\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\Public\Documents\ESTsoft\CreatorTemp\pip-wheel-fgzdpaca'
       cwd: C:\Users\Public\Documents\ESTsoft\CreatorTemp\pip-install-x2xpk4ne\scikit-surprise_73cc58ddbebb4279b6652d98bb1d3e37\
  Complete output (55 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.8
  creating build\lib.win-amd64-3.8\surpri

In [2]:
import pandas as pd # raw dataset
from surprise import SVD, accuracy # SVD model, 평가
from surprise import Reader, Dataset # SVD model의 dataset

rating = pd.read_csv("movie_rating.csv")
rating.head()

Lisa       6
Mick       6
Gene       6
Jack       5
Claudia    5
Toby       3
Name: critic, dtype: int64

In [3]:
rating['critic'].value_counts()

Lisa       6
Mick       6
Gene       6
Jack       5
Claudia    5
Toby       3
Name: critic, dtype: int64

In [4]:
rating['title'].value_counts()

Superman     6
Snakes       6
You Me       6
The Night    5
Just My      4
Lady         4
Name: title, dtype: int64

In [5]:
tab = pd.crosstab(rating['critic'], rating['title'])
tab

title,Just My,Lady,Snakes,Superman,The Night,You Me
critic,,,,,,
Claudia,1,0,1,1,1,1
Gene,1,1,1,1,1,1
Jack,0,1,1,1,1,1
Lisa,1,1,1,1,1,1
Mick,1,1,1,1,1,1
Toby,0,0,1,1,0,1


In [8]:
rating_g = rating.groupby(['critic','title'])
rating_g.sum().unstack() # 행렬구조로 변환

rating                                      
title   Just My Lady Snakes Superman The Night You Me
critic                                               
Claudia     3.0  NaN    3.5      4.0       4.5    2.5
Gene        1.5  3.0    3.5      5.0       3.0    3.5
Jack        NaN  3.0    4.0      5.0       3.0    3.5
Lisa        3.0  2.5    3.5      3.5       3.0    2.5
Mick        2.0  3.0    4.0      3.0       3.0    2.0
Toby        NaN  NaN    4.5      4.0       NaN    1.0

## Toby가 미관람한 영화 : 'Just My', 'Lady', 'The Night'

In [11]:
reader = Reader(rating_scale = (1,5))  # 평점 범위
data = Dataset.load_from_df(df=rating, reader = reader)
data # rating이라는 데이터프레임은 reader(1~5)의 평점 범위를 가진다.

In [14]:
train = data.build_full_trainset() # 훈련셋
test = train.build_testset() # 검정셋

help(SVD)
model = SVD(n_factors=100, n_epochs=20, random_state=123)
model.fit(train) # model 생성

Help on class SVD in module surprise.prediction_algorithms.matrix_factorization:

class SVD(surprise.prediction_algorithms.algo_base.AlgoBase)
 |  SVD(n_factors=100, n_epochs=20, biased=True, init_mean=0, init_std_dev=0.1, lr_all=0.005, reg_all=0.02, lr_bu=None, lr_bi=None, lr_pu=None, lr_qi=None, reg_bu=None, reg_bi=None, reg_pu=None, reg_qi=None, random_state=None, verbose=False)
 |  
 |  The famous *SVD* algorithm, as popularized by `Simon Funk
 |  <http://sifter.org/~simon/journal/20061211.html>`_ during the Netflix
 |  Prize. When baselines are not used, this is equivalent to Probabilistic
 |  Matrix Factorization :cite:`salakhutdinov2008a` (see :ref:`note
 |  <unbiased_note>` below).
 |  
 |  The prediction :math:`\hat{r}_{ui}` is set as:
 |  
 |  .. math::
 |      \hat{r}_{ui} = \mu + b_u + b_i + q_i^Tp_u
 |  
 |  If user :math:`u` is unknown, then the bias :math:`b_u` and the factors
 |  :math:`p_u` are assumed to be zero. The same applies for item :math:`i`
 |  with :math:`b_i

In [16]:
user_id = 'Toby' # 추천대상자
item_ids = ['The Night', 'Just My', 'Lady'] # 추천 대상 영화
actual_rating = 0 # 실제 평점

for item_id in item_ids :
    print(model.predict(user_id, item_id, actual_rating))

user: Toby       item: The Night  r_ui = 0.00   est = 3.30   {'was_impossible': False}
user: Toby       item: Just My    r_ui = 0.00   est = 2.88   {'was_impossible': False}
user: Toby       item: Lady       r_ui = 0.00   est = 3.27   {'was_impossible': False}
